In [3]:
# -*- coding: utf-8 -*-
# 埼玉千葉神奈川記事反映本番用
import requests
import json
from wordpress_xmlrpc.methods import media
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods import media, posts
from wordpress_xmlrpc.compat import xmlrpc_client
from wordpress_xmlrpc import Client, WordPressPost
from PIL import Image, ImageDraw, ImageFont
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import os
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import Select
import urllib
from bs4 import BeautifulSoup
import re
import csv
import codecs
import requests
import urllib.request as req
import glob
import json
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as ticker
from matplotlib import rcParams
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from datetime import timedelta
import datetime
import os
from dotenv import load_dotenv
from selenium.webdriver.common.by import By

print('ライブラリの読み込み完了')

# .envファイルの内容を読み込見込む
load_dotenv()



def get_concat_h_blank(im1, im2, color=(27, 44, 74)):
    dst = Image.new('RGB', (im1.width + im2.width, max(im1.height, im2.height)), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst


def get_concat_v(im1, im2, color=(0, 0, 0)):
    dst = Image.new('RGB', (max(im1.width, im2.width), im1.height + im2.height), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst


def get_concat_v_blank(im1, im2, color=(0, 0, 0)):
    dst = Image.new('RGB', (max(im1.width, im2.width), im1.height + im2.height), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst


def upload_image(in_image_file_name, out_image_file_name):
    if os.path.exists(in_image_file_name):
        with open(in_image_file_name, 'rb') as f:
            binary = f.read()

        data = {
            "name": out_image_file_name,
            "type": 'image/png',
            "overwrite": True,
            "bits": binary
        }

        media_id = wp.call(media.UploadFile(data))['id']
        print(in_image_file_name.split('/')
              [-1], 'Upload Success : id=%s' % media_id)
        return media_id
    else:
        print(in_image_file_name.split('/')[-1], 'NO IMAGE!!')

def post_line(message):
    url = "https://notify-api.line.me/api/notify"
    token = os.environ['LINE_TOKEN']
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    #print('image_path',image_path)
    #files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload) 



ライブラリの読み込み完了


In [4]:
WORDPRESS_ID = os.environ['WORDPRESS_ID']
WORDPRESS_PW = os.environ['WORDPRESS_PW']
WORDPRESS_URL = os.environ['WORDPRESS_URL']
wp = Client(WORDPRESS_URL, WORDPRESS_ID, WORDPRESS_PW)

In [ ]:
#recommend_image2()

#upload_image(save_image_jpg, save_image_jpg.replace('image/全媒体/',''))
sumnail_path = win_path + f'kantou_syuzaireport_{tomorrow_url}.jpg'


wp = Client(WORDPRESS_URL, WORDPRESS_ID, WORDPRESS_PW)

# for i in range(1,4):
#     imgPath = win_path+f'heatmap/tokyo_heatmap_{tenpo_name}_{date_list[0]}_{i}.jpg'
#     upload_image(imgPath, imgPath)
#     time.sleep(1)


# import


# Picture file name & Upload
imgPath = sumnail_path
media_id = upload_image(imgPath, imgPath)

# Blog Title

title = f"【{string_date}】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 "
# Blog Content (html)
body = f'''
{wp_completed_text_compted}
'''

# publish or draft
status = "publish"

# Category keyword
cat1 = '埼玉・神奈川・千葉 スロットイベントまとめ・オススメ店舗'
cat2 = ''
cat3 = ''

# Tag keyword
tag1 = f'{string_date[-4]}のつく日'
tag2 = f'イベント取材まとめ'
tag3 = f'スロット'
tag4 = f'{tomorrow_url}'

slug = f"kantou_syuzaireport_{tomorrow_url}"

# Post
post = WordPressPost()
post.title = title
post.content = body
post.post_status = status
post.terms_names = {
    "category": [cat1],
    "post_tag": [tag1, tag2, tag3, tag4],
}
post.slug = slug

# Set eye-catch image

post.thumbnail = media_id

# Post Time
#post.date = datetime.datetime.now() - datetime.timedelta(hours=9)

wp.call(NewPost(post))
print('記事書き込み完了')
post_line(f'{tomorrow}分がブログ投稿完了')
# except Exception as e:
#     post_line(f'{tomorrow}分ブログ投稿失敗\n{e}')